# MuhRec in python demonstration

In [ ]:
import sys, os
sys.path.insert(0, "/home/kaestner/git/install/lib")
sys.path.insert(0, "/home/kaestner/git/scripts/python/")
if 'LD_LIBRARY_PATH' not in os.environ:
    os.environ['LD_LIBRARY_PATH'] = '/home/kaestner/git/install/lib'
   # os.execv(sys.argv[0], sys.argv)




In [ ]:
import numpy as np
import muhrectomo as mt
import matplotlib.pyplot as plt
import imgalg
import amglib.readers as io
import amglib.imageutils as amg

In [ ]:
import importlib
importlib.reload(mt)
importlib.reload(io)
importlib.reload(amg)
importlib.reload(imgalg)

## Create a reconstructor object

In [ ]:
recon = mt.Reconstructor(mt.bpMultiProj)

print("Created reconstructor :",recon.name())

## Reconstruction workflow

### Numerical dummy
This dummy is only to test the connection and that the reconstructor starts at all. Only nonsense data.

In [ ]:
Nproj = 10
proj = np.ones([Nproj,256,256])

In [ ]:
args = {"angles" : np.linspace(0,180,num=Nproj), 
        "weights" : np.ones(Nproj)}

In [ ]:
recon.configure({   "center" : 50, 
                    "resolution" : 0.135
                })

In [ ]:
recon.process(proj,args)

vol = recon.volume()

In [ ]:
plt.imshow(vol[0])

### The wood data
The wood data is the data I use a lot for demos, tests, and tutorials. It a piece of petrified wood [DOI](http://dx.doi.org/10.17632/g5snr785xy.2). 

#### Load projection data

In [ ]:
ob = io.readImages('/home/kaestner/data/wooddata/projections/ob_{0:04d}.tif',1,5,averageStack=True).mean(axis=0)
dc = io.readImages('/home/kaestner/data/wooddata/projections/dc_{0:04d}.tif',1,5,averageStack=True).mean(axis=0)

In [ ]:
proj = io.readImages('/home/kaestner/data/wooddata/projections/wood_{0:04d}.tif',1,626) # This takes a while

In [ ]:
proj.dtype

In [ ]:
fig,ax = plt.subplots(1,3,figsize=[15,7])
ax[0].imshow(dc)
ax[1].imshow(ob)
ax[2].imshow(proj[0])


## Preprocessing
Here the projections needs to be prepared for reconstruction. Typical operations are 
- Cropping
- Normalization (possibly with scattering correction)
- Spot cleaning
- Ring cleaning

### Normalize - the kipl way

In [ ]:
import imgalg 

In [ ]:
help(imgalg)

In [ ]:
norm = imgalg.NormalizeImage(True) # True for use logarithm

In [ ]:
norm.usingLog()

In [ ]:
norm = imgalg.NormalizeImage(True) # True for use logarithm
norm.setReferences(ob,dc)
cproj = proj.copy()
norm.process(cproj)

## Normalize - the python way

In [ ]:
nproj = amg.normalizeImage(img=proj, ob=ob, dc=dc, neglog=True)

In [ ]:
nproj.dtype

In [ ]:
fig,axes=plt.subplots(1,5,figsize=(15,4))

for idx,ax in enumerate(axes) :
    ax.imshow(cproj[idx*20]-nproj[idx*20])

In [ ]:
print(nproj.shape,cproj.shape)

In [ ]:
fig,axes=plt.subplots(1,3,figsize=(15,4))
a0=axes[0].imshow(cproj[:,400,:], vmin=-0.1,vmax=1); axes[0].set_title('cproj')
fig.colorbar(a0,ax=axes[0])
a1=axes[1].imshow(nproj[:,400,:], vmin=-0.1,vmax=1); axes[1].set_title('nproj')
fig.colorbar(a1,ax=axes[1])
a2=axes[2].imshow(cproj[:,400,:]-nproj[:,400,:]); axes[2].set_title('diff')
fig.colorbar(a2,ax=axes[2])

cproj.shape

#### Prepare and run the back-projection

In [ ]:
ce = imgalg.TomoCenter()

ce.estimate(cproj[0,:,:],cproj[312,:,:],imgalg.centerLeastSquare,True)

In [ ]:
Nproj = cproj.shape[0]
# Information per projection
args = {"angles"  : np.linspace(0,360,num=Nproj), 
        "weights" : np.ones(Nproj)/Nproj
       }

# Geometry information
recon.configure({   "center" : 305, 
                    "resolution" : 0.1
                })
print("have",Nproj,"projections")

In [ ]:
recon.process(cproj[:,500:510,:].astype('float64'),args) # Reconstruct a part of the slices (32 slices here)
del vol
vol = recon.volume() # Retrieve the reconstructed volume

In [ ]:
plt.imshow(vol[0])
plt.colorbar()
plt.savefig('recon.png',dpi=300)

In [ ]:
import ipyvolume as ipv

In [ ]:
ipv.quickvolshow(vol)

In [ ]:
print(nproj.flags)
print(cproj.flags)

In [ ]:
a=cproj[:,500:510,:]
b=nproj[:,500:510,:]

# Spotcleaning tests

In [ ]:
msc = imgalg.MorphSpotClean()

In [ ]:
strip= cproj[1,500:600,:]
msc.setCleanMethod(detectionMethod=imgalg.MorphDetectAllSpot, cleanMethod=imgalg.MorphCleanReplace);
d=msc.detectionImage(strip,True)
fig, ax = plt.subplots(1,3,figsize=(15,3))
ax[0].imshow(strip)
ax[1].imshow(d['bright'])
phist,pbins=np.histogram(d.ravel(), bins=500)
ax[2].plot(bins[:-1],np.cumsum(hist)/hist.sum())

In [ ]:
strip= cproj[1,500:600,:]
msc.setCleanMethod(detectionMethod=imgalg.MorphDetectHoles, cleanMethod=imgalg.MorphCleanReplace);
dh=msc.detectionImage(strip,True)


In [ ]:
dh

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(15,3))
ax[0].imshow(strip)
ax[1].imshow(dh[])
hhist,hbins=np.histogram(dh.ravel(), bins=500)
ax[2].plot(bins[:-1],np.cumsum(hhist)/hhist.sum())

In [ ]:
strip= cproj[1,500:600,:].copy()
orig = cproj[1,500:600,:]
msc.process(strip,th=[0.02, 0.02],sigma=[0.001, 0.001])
fig, ax = plt.subplots(1,3,figsize=(15,3))
ax[0].imshow(orig)
ax[1].imshow(strip)
ax[2].imshow(strip-orig)

# Ring cleaning

In [ ]:
sino=cproj[:,500,:]
sorig=sino.copy()
plt.imshow(sino)

In [ ]:
wrc = imgalg.StripeFilter([sino.shape[1],sino.shape[0]],'daub17',3,0.01)

In [ ]:
wrc.process(sino,imgalg.VerticalComponentFFT)

In [ ]:
plt.imshow(sino-sorig,vmin=-0.0005,vmax=0.0005)

In [ ]:
d=sino-sorig
plt.hist(d.ravel(),bins=500);